In [ ]:
pip install fcit

In [ ]:
pip install pycid

In [ ]:
pip install causalnex

In [ ]:
!git clone https://github.com/pchlenski/gda_project/
!cd gda_project && git pull

In [6]:
import pandas as pd
from gda_project.utils.embed import *
from gda_project.utils.scm_large import *
from gda_project.utils.scm import *
from gda_project.utils.independence import *
import networkx as nx
import rpy2
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
import warnings
from causalnex.structure import StructureModel
from causalnex.structure.notears import from_pandas
from causalnex.plots import plot_structure, NODE_STYLE, EDGE_STYLE
import networkx as nx

warnings.filterwarnings("ignore")  # silence warnings

sm = StructureModel()

## Student dataset

In [8]:
X = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/GDA/Final Project/student-mat.csv', delimiter=';')
X.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,4,6,10,10


In [9]:
X_numeric = X.copy()
non_numeric_columns = list(X_numeric.select_dtypes(exclude=[np.number]).columns)
le = LabelEncoder()

for col in non_numeric_columns:
    X_numeric[col] = le.fit_transform(X_numeric[col])

X_numeric.head(5)

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,0,0,18,1,0,0,4,4,0,4,...,4,3,4,1,1,3,6,5,6,6
1,0,0,17,1,0,1,1,1,0,2,...,5,3,3,1,1,3,4,5,5,6
2,0,0,15,1,1,1,1,1,0,2,...,4,3,2,2,3,3,10,7,8,10
3,0,0,15,1,0,1,4,2,1,3,...,3,2,2,1,1,5,2,15,14,15
4,0,0,16,1,0,1,3,3,2,2,...,4,3,2,1,2,5,4,6,10,10


In [21]:
idx = np.array([1,2,3])
X_numeric.index

RangeIndex(start=0, stop=395, step=1)

In [19]:
# Non-causal performance

idx = random.sample(range(395),80)
test_X = X_numeric.iloc[idx]
train_X = X_numeric
train_X = train_X.drop(labels=idx,axis=0)
test_Y = test_X.G3
train_Y = train_X.G3
train_X = train_X.drop(labels='G3',axis=1)
test_X = test_X.drop(labels='G3',axis=1)
train_X = run_pca(train_X,n_dims=5,n_neighbors=5)
test_X = run_pca(test_X,n_dims=5,n_neighbors=5)

n_estimators=10
max_depth=50
cf1 = RandomForestClassifier(n_estimators=n_estimators,max_depth=50)
cf1.fit(train_X,train_Y)

validation_error_erm = (cf1.predict(test_X)-test_Y).abs().sum()/80
# print("Non-causal prediction validation error: ", validation_error)

In [20]:
# Causal performance

train_X = X_numeric.loc[X_numeric.school==0]
test_X = X_numeric.loc[X_numeric.school==1]
test_Y = test_X.G3
train_Y = train_X.G3
train_X = train_X.drop(labels='G3',axis=1)
test_X = test_X.drop(labels='G3',axis=1)
train_X = run_pca(train_X,n_dims=5,n_neighbors=5)
test_X = run_pca(test_X,n_dims=5,n_neighbors=5)

n_estimators=10
max_depth=50
cf1 = RandomForestClassifier(n_estimators=n_estimators,max_depth=50)
cf1.fit(train_X,train_Y)

validation_error_irm = (cf1.predict(test_X)-test_Y).abs().sum()/46
# print("Causal prediction validation error: ", validation_error)

In [21]:
print('ERM loss: ', validation_error_erm)
print('IRM loss: ', validation_error_irm)
print("Conclusion: PCA does not retrieve causal features")

ERM loss:  1.375
IRM loss:  5.326086956521739
Conclusion: PCA does not retrieve causal features
